In [7]:
import xml.etree.ElementTree as ET
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
import re
import numpy as np

In [8]:
# Parametros para funciones generadoras
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
TEMPLATE_MA = 'template-ma.ma'

def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

In [42]:
############### GENERATE .H AND .CPP ###############
def generateF(template, fType, stock, variables, formula):
    # Nota: explicacion de parametros que toma generateF ==>
    # template : nombre del template a utilizar (debe haber un .cpp y un .h con este nombre)
    # fType : 'fPlus' o 'fMinus' son los dos tipos de funciones que se pueden declarar
    # stock : nombre del 'stock' asociado a esta funcion (el nombre de la variable que esta siendo integrada)
    # variables : nombres de las variables de estado utilizadas en la funcion
    # formula : formula de la outputFunction, que utiliza las variables de estado del atomico
    
    # Template names
    templateH   = template + '.h'
    templateCPP = template + '.cpp'
    
    # Derived parameters for template
    stockName = fType + stock[0].upper() + stock[1:]
    stockNameConstant = stockName.upper()
    setVariables  = list(map(lambda var : 'isSet' + var[0].upper() + var[1:], variables))
    inPorts       = list(map(lambda var : 'in' + var[0].upper() + var[1:], variables))
    context = {
        'stockName' : stockName,
        'stockNameConstant' : stockNameConstant,
        'variables' : variables,
        'setVariables' : setVariables,
        'inPorts' : inPorts,
        'formula' : formula
    }
    hFile = render_template(templateH, context)
    cppFile = render_template(templateCPP, context)
    return {"h" : hFile, "cpp" : cppFile}

def generateReg(template, stockNames):
    stockNamesConstant = list(map(lambda var : var.upper(), stockNames))
    templateCPP = template + '.cpp'
    context = { 'stockNames' : stockNames, 'stockNamesConstant' : stockNamesConstant }
    cppFile = render_template(templateCPP, context)
    return cppFile

def generateFiles(folder, atomics):
    for atomic in atomics:
        f = generateF(atomic[0], atomic[1], atomic[2], atomic[3], atomic[4])
        hFile   = f['h']
        cppFile = f['cpp']
        # Genero hFile y cppFile
        with open(folder + '/' + atomic[1] + atomic[2][0].upper() + atomic[2][1:] + '.h', 'w') as f:
            f.write(hFile)
        with open(folder + '/' + atomic[1] + atomic[2][0].upper() + atomic[2][1:] + '.cpp', 'w') as f:
            f.write(cppFile)
    # Genero 'reg.cpp'
    atomicNames = [atomic[1] + atomic[2][0].upper() + atomic[2][1:] for atomic in atomics]
    regFile = generateReg('reg', atomicNames)
    with open(folder + '/reg.cpp', 'w') as f:
        f.write(regFile)

In [96]:
def devsml2cdpp(archivoDevsml, archivoMa, srcCPP):
    tree = ET.parse(archivoDevsml)
    root = tree.getroot()

    coupled = root.find('scenario').find('coupled')
    inputPorts = coupled.find('inputs').findall('port')
    outputPorts = coupled.find('outputs').findall('port')
    atomicos = coupled.find('components').findall('atomicRef')
    links_internal = coupled.find('internal_connections').findall('connection')
    links_external = coupled.find('external_input_connections').findall('connection')

    # Atomicos
    integradores, fms, fps, fts, ctes = [], [], [], [], []
    for atomico in atomicos:
        if atomico.get('model') == 'QSS1':
            integradores.append(atomico)
        if atomico.get('model') == 'Ftot':
            fts.append(atomico)
        if atomico.get('model') == 'Fminus':
            fms.append(atomico)
        if atomico.get('model') == 'Fplus':
            fps.append(atomico)
        if atomico.get('model') == 'Cte':
            ctes.append(atomico)

    # Conexiones internas
    links_internal_modelo = []
    links_internal_ctes, links_internal_acoplado_minimal = {}, {}

    for integrador in integradores:
        nombreVariable = integrador.get('name').replace(' Integrator', '')
        links_internal_acoplado_minimal[nombreVariable] = []
    for cte in ctes:
        links_internal_ctes[cte.get('name')] = []

    for link in links_internal:
        for nombreVar, ls in links_internal_acoplado_minimal.iteritems():
            # Links modelo acoplado minimal de cada integrador
            if link.get('component_from') in map(lambda x : x + ' ' + nombreVar, ['fm', 'fp', 'ft']):
                links_internal_acoplado_minimal[nombreVar].append(link)

    for link in links_internal:
        # Links a atomicos Cte's que llegan de afuera
        if link.get('component_from') in [cte.get('name') for cte in ctes]:
            links_internal_ctes[link.get('component_from')].append(link)
        # Links importantes del modelo
        if 'Integrator' in link.get('component_from'):
            links_internal_modelo.append(link)

    # Transformo los objetos tipo Element en diccionarios, formato mas accesible para acceder a los datos en jinja2
    inputPorts_ = {}
    for port in inputPorts:
        name = 'in' + port.get('name').replace(' ', '')
        inputPorts_[port.get('name').replace(' ', '')] = name
    outputPorts_ = {}
    for port in outputPorts:
        name = 'out' + port.get('name').replace(' ', '')
        outputPorts_[name] = None    

    integradores_ = {}
    for integrador in integradores:
        name = integrador.get('name').replace(' ', '')
        name_lower = name[0].lower() + name[1:]
        attribs = {'name_lower' : name_lower}
        for param in integrador.findall('parameter'):
            attribs[param.get('name')] = param.get('value')
        integradores_[name] = attribs
    fts_ = {}
    for ft in fts:
        name = ft.get('name').replace(' ', '')
        fts_[name] = None
    fps_ = {}
    for fp in fps:
        name = fp.get('name').split('-')[0].replace(' ', '')
        fps_[name] = {
            'function' : fp.find('parameter').get('value').replace(' ', ''),
            'stock' : name[2:]
        }
    fms_ = {}
    for fm in fms:
        name = fm.get('name').split('-')[0].replace(' ', '')
        fms_[name] = {
            'function' : fm.find('parameter').get('value').replace(' ', ''),
            'stock' : name[2:]
        }
    ctes_ = {}
    for cte in ctes:
        name = cte.get('name').replace(' ', '')
        attribs = {}
        name_lower = name[0].lower() + name[1:]
        attribs['name_lower'] = name_lower
        attribs['ports'] = [port.get('type') + port.get('name')[0].upper() + port.get('name')[1:]  
                            for port in cte.findall('port')]
        attribs['value'] = cte.find('parameter').get('value')
        ctes_[name] = attribs

    # TODO 
    for cteName, elems in links_internal_ctes.iteritems():
        new_elems = []
        for conn in elems:
            attr = {}
            attr['component_from'] = ctes_[conn.get('component_from').replace(' ', '')]['name_lower']
            attr['port_from'] = conn.get('port_from')
            attr['component_to'] = conn.get('component_to').split('-')[0].replace(' ', '')
            attr['port_to'] = 'in' + conn.get('port_to').replace(' ', '')
            new_elems.append(attr)
        links_internal_ctes[cteName] = new_elems

    # Aca van juntos los fm's, fp's y ft's (component_to puede ser ft{Integrador} o {Integrador})
    for intName, elems in links_internal_acoplado_minimal.iteritems():
        new_elems = []
        for conn in elems:
            attr = {}
            # aca, la componente figura sin el '-'. OK
            attr['component_from'] = conn.get('component_from').replace(' ', '')
            
            comp_to = conn.get('component_to').replace(' ', '')
            attr['component_to'] = comp_to[0].lower() + comp_to[1:] 
            attr['port_from'] = conn.get('port_from')
            attr['port_to'] = conn.get('port_to')
            new_elems.append(attr)
        links_internal_acoplado_minimal[intName] = new_elems

    links_internal_modelo_ = []
    for link in links_internal_modelo:
        new_link = {}
        new_link['component_from'] = integradores_[link.get('component_from').replace(' ', '')]['name_lower']
        new_link['component_to'] = link.get('component_to').split('-')[0].replace(' ', '')
        new_link['port_from'] = link.get('port_from')
        new_link['port_to'] = 'in' + link.get('port_to').replace(' ', '')
        links_internal_modelo_.append(new_link)
    
    context={
        'integradores_' : integradores_,
        'fts_' : fts_,
        'fps_' : fps_,
        'fms_' : fms_,
        'ctes_': ctes_,
        'inputPorts_' : inputPorts_,
        'outputPorts_' : outputPorts_,
        'links_internal_ctes' : links_internal_ctes,
        'links_internal_acoplado_minimal' : links_internal_acoplado_minimal,
        'links_internal_modelo_' : links_internal_modelo_
    }
    
    #####
    # Genero parametros para el generador de los .h y .cpp
    fileGenParams = []
    for fm, attr in fms_.iteritems():
        stock = attr['stock'][0].lower() + attr['stock'][1:]
        function = attr['function']
        params = []
        for intName, _ in integradores_.iteritems():
            if intName in function:
                function = function.replace(intName, intName[0].lower() + intName[1:])
                params.append(intName[0].lower() + intName[1:])
        for cteName, _ in ctes_.iteritems():
            if cteName in function:
                function = function.replace(cteName, cteName[0].lower() + cteName[1:])
                params.append(cteName[0].lower() + cteName[1:])
        fileGenParams.append(['Fpm', 'Fminus', stock, params, function])
    for fp, attr in fps_.iteritems():
        stock = attr['stock'][0].lower() + attr['stock'][1:]
        function = attr['function']
        params = []
        for intName, _ in integradores_.iteritems():
            if intName in function:
                function = function.replace(intName, intName[0].lower() + intName[1:])
                params.append(intName[0].lower() + intName[1:])
        for cteName, _ in ctes_.iteritems():
            if cteName in function:
                function = function.replace(cteName, cteName[0].lower() + cteName[1:])
                params.append(cteName[0].lower() + cteName[1:])
        fileGenParams.append(['Fpm', 'Fplus', stock, params, function])
    
    #####
    # Guardar el .ma en output
    with open(archivoMa, 'w') as f:
        f.write(render_template(TEMPLATE_MA,context))
    # Guardar el .h y .cpp en output
    generateFiles(srcCPP, fileGenParams)

In [97]:
# Generador de archivo .ma
archivoDevsml = 'teacup/teacup-devsml-traduccion.xml'
archivoMa = 'teacup/teacup-ma-traduccion.ma'
devsml2cdpp(archivoDevsml, archivoMa, 'teacup/src')

archivoDevsml = 'sir/sir-devsml-traduccion.xml'
archivoMa = 'sir/sir-ma-traduccion.ma'
devsml2cdpp(archivoDevsml, archivoMa, 'sir/src')